In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


train_dir = r"C:\Users\admin\Desktop\SIH\brain tumor\input\input"
val_dir   = r"C:\Users\admin\Desktop\SIH\brain tumor\input\input"

img_size = (96, 96)   
batch_size = 4        


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir, target_size=img_size, batch_size=batch_size, class_mode="categorical"
)
val_gen = val_datagen.flow_from_directory(
    val_dir, target_size=img_size, batch_size=batch_size, class_mode="categorical", shuffle=False
)


y_train_labels = train_gen.classes
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_labels),
    y=y_train_labels
)
class_weights = dict(enumerate(class_weights))
print("Class Weights:", class_weights)


base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(96,96,3))
base_model.trainable = False  

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)  
x = layers.Dense(128, activation="relu")(x) 
x = layers.Dropout(0.4)(x)
predictions = layers.Dense(train_gen.num_classes, activation="softmax")(x)

model = models.Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss="categorical_crossentropy",
              metrics=["accuracy", 
                       tf.keras.metrics.Precision(name="precision"),
                       tf.keras.metrics.Recall(name="recall"),
                       tf.keras.metrics.AUC(name="auc")])


cb = [
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint("best_model.h5", save_best_only=True)
]

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    class_weight=class_weights,
    callbacks=cb
)

base_model.trainable = True
for layer in base_model.layers[:-30]:   
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  
              loss="categorical_crossentropy",
              metrics=["accuracy", "precision", "recall", "auc"])

history_fine = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    class_weight=class_weights,
    callbacks=cb
)

val_loss, val_acc, val_prec, val_rec, val_auc = model.evaluate(val_gen)
print(f"\nValidation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Validation Precision: {val_prec:.4f}")
print(f"Validation Recall: {val_rec:.4f}")
print(f"Validation AUC: {val_auc:.4f}")


y_true = val_gen.classes
y_pred_probs = model.predict(val_gen)
y_pred = np.argmax(y_pred_probs, axis=1)

print("\nClassification Report:\n")
print(classification_report(y_true, y_pred, target_names=list(val_gen.class_indices.keys())))


cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=val_gen.class_indices.keys(),
            yticklabels=val_gen.class_indices.keys())
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

for i in range(5):
    probs = y_pred_probs[i]
    top_class = np.argmax(probs)
    confidence = np.max(probs) * 100
    print(f"Image {i+1}: Predicted {list(val_gen.class_indices.keys())[top_class]} "
          f"({confidence:.2f}% confidence)")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\admin\\Desktop\\SIH\\brain tumor\\input\\input'